In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from scipy.optimize import minimize
from deap import base, creator, tools, algorithms
from sklearn.linear_model import LinearRegression

# 加载数据
data = pd.read_csv("E:\\Mathematics_Modeling_study\\2025MCM\\Juneau_field_data.csv")

# 数据规范化
scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(data.iloc[:, 1:])

# 定义决策变量和目标变量
X = normalized_data[:, :8]  # 决策变量
Y = normalized_data[:, 8:]  # 目标变量

# 构建线性回归模型
models = []
for i in range(Y.shape[1]):
    model = LinearRegression().fit(X, Y[:, i])
    models.append(model)

# 定义目标函数
def objective_function(x):
    y_predictions = [model.predict([x])[0] for model in models]
    objective_value = np.sum(y_predictions)
    return (objective_value,)

# 定义约束条件
def constraint_function(x):
    return np.sum(x) - 1

# 直接搜索法
x0 = np.random.rand(8)
result = minimize(objective_function, x0, constraints={'type': 'eq', 'fun': constraint_function})
print("Direct Search Result:", result.x)

# 遗传算法
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("attr_float", np.random.rand)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=8)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("mate", tools.cxBlend, alpha=0.5)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", objective_function)

population = toolbox.population(n=50)
algorithms.eaSimple(population, toolbox, cxpb=0.5, mutpb=0.2, ngen=40, verbose=True)

best_individual = tools.selBest(population, 1)[0]
print("Genetic Algorithm Best Individual:", best_individual)

# 结果表格
decision_variables = {
    "Genetic Algorithm": best_individual,
    "Direct Search": result.x
}

sub_goals = {
    "Genetic Algorithm": objective_function(best_individual)[0],
    "Direct Search": objective_function(result.x)[0]
}

print("Decision Variables:")
print(pd.DataFrame(decision_variables))

print("\nSub-Goals:")
print(pd.DataFrame(sub_goals, index=["Objective Value"]))

Direct Search Result: [ 1.48715289e+08  1.47613808e+08 -9.84399049e+07 -2.53180845e+07
 -9.52243427e+07 -1.75883300e+08  7.33802290e+07  2.51563076e+07]
gen	nevals
0  	50    
1  	33    
2  	38    
3  	25    
4  	33    
5  	23    
6  	34    
7  	32    
8  	39    
9  	32    
10 	32    
11 	37    
12 	26    
13 	32    
14 	27    
15 	33    
16 	30    
17 	28    
18 	30    
19 	35    
20 	26    
21 	25    
22 	25    
23 	36    
24 	32    
25 	35    
26 	33    
27 	29    
28 	21    
29 	28    
30 	28    
31 	30    
32 	35    
33 	29    
34 	32    
35 	36    
36 	29    
37 	28    
38 	27    
39 	38    
40 	34    
Genetic Algorithm Best Individual: [20.968040400493628, 13.759344832092552, -1.5965350412311659, -0.7758366319012115, -5.469871461093128, -9.108764568739963, 9.385203390607222, 4.827365831689709]
Decision Variables:
   Genetic Algorithm  Direct Search
0          20.968040   1.487153e+08
1          13.759345   1.476138e+08
2          -1.596535  -9.843990e+07
3          -0.775837  -2.